In [ ]:
#pip install prettyprinter

In [ ]:
#pip install ruamel.yaml

In [1]:
import argparse
import json
import os
import pickle
import random
import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoConfig

from data_utils import (YamlConfigManager, WOSDataset, get_examples_from_dialogues, load_dataset,
                        set_seed, custom_to_mask, custom_get_examples_from_dialogues, custom_load_dataset)

from evaluation import _evaluation
from inference import inference
from model import TRADE, masked_cross_entropy_for_value
from preprocessor import TRADEPreprocessor
from prettyprinter import cpprint

from pathlib import Path
import glob
import re

import wandb
import time

from torch.cuda.amp import GradScaler, autocast

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
cfg = YamlConfigManager('./config.yml', 'base').values
cpprint(cfg)

easydict.EasyDict({
    'data_dir': '../input/data/train_dataset',
    'model_dir': 'results',
    'train_batch_size': 4,
    'eval_batch_size': 8,
    'learning_rate': 3e-05,
    'adam_epsilon': 1e-08,
    'max_grad_norm': 1.0,
    'num_train_epochs': 30,
    'warmup_ratio': 0.0,
    'random_seed': 42,
    'n_gate': 5,
    'teacher_forcing_ratio': 0.5,
    'model_name_or_path': 'dsksd/bert-ko-small-minimal',
    'proj_dim': 'None',
    'tag': ['trade'],
    'use_kfold': False,
    'num_k': 0,
    'val_ratio': 0.1,
    'scheduler': 'Linear',
    'mask': True
})


In [4]:
# Get current learning rate
def get_lr(scheduler):
    return scheduler.get_last_lr()[0]

In [5]:
# random seed 고정
set_seed(cfg.random_seed)

# Data Loading
train_data_file = f"{cfg.data_dir}/train_dials.json"
slot_meta = json.load(open(f"{cfg.data_dir}/slot_meta.json"))
# train_data, dev_data, dev_labels = load_dataset(train_data_file, cfg.val_ratio)
train_data, dev_data, dev_labels = custom_load_dataset(train_data_file, cfg.val_ratio, k=8)

train_examples = custom_get_examples_from_dialogues(
    train_data, user_first=False, dialogue_level=False
)
dev_examples = custom_get_examples_from_dialogues(
    dev_data, user_first=False, dialogue_level=False
)

 17%|█▋        | 1086/6300 [00:00<00:00, 10859.75it/s]

k  8
[ 149 4765 4335 2340 4677 6176 5227 6556 5449  657]


100%|██████████| 700/700 [00:00<00:00, 2595.12it/s]


In [6]:
train_examples[0]

DSTInputExample(guid='snowy-hat-8324:관광_식당_11-0', context_turns=[], current_turn=['', ' # ', '서울 중앙에 있는 박물관을 찾아주세요', ' * '], label=['관광-종류-박물관', '관광-지역-서울 중앙'])

In [7]:
# Define Preprocessor
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name_or_path)

# Dealing with long texts The maximum sequence length of BERT is 512.
processor = TRADEPreprocessor(slot_meta, tokenizer, max_seq_length=512, n_gate=cfg.n_gate)

In [8]:
# Extracting Featrues
# cpprint('Extracting Features...')
# train_features = processor.sep_custom_convert_examples_to_features(train_examples)
# dev_features = processor.sep_custom_convert_examples_to_features(dev_examples)

In [9]:
# # 전체 train data InputFeatur 저장
# with open('custom_train_features.txt', 'wb') as f:
#     pickle.dump(train_features, f)
# with open('custom_dev_features.txt', 'wb') as f:
#     pickle.dump(dev_features, f)

In [10]:
# 저장된 파일 사용
with open('custom_train_features.txt', 'rb') as f:
    train_features = pickle.load(f)
with open('custom_dev_features.txt', 'rb') as f:
    dev_features = pickle.load(f)

In [11]:
# Slot Meta tokenizing for the decoder initial inputs
tokenized_slot_meta = []
for slot in slot_meta:
    tokenized_slot_meta.append(
        tokenizer.encode(slot.replace("-", " "), add_special_tokens=False)
    )

In [12]:
# Model 선언
config = AutoConfig.from_pretrained('dsksd/bert-ko-small-minimal')
config.model_name_or_path = 'dsksd/bert-ko-small-minimal'
config.n_gate = cfg.n_gate
config.proj_dim = None

model = TRADE(config, tokenized_slot_meta)

model.to(device)
print("Model is initialized")

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Model is initialized


In [13]:
# --wandb initialize with configuration
wandb.init(project='DST', tags=cfg.tag, config=cfg)

wandb: Currently logged in as: taepd (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
train_data = WOSDataset(train_features)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(
    train_data,
    batch_size=cfg.train_batch_size,
    sampler=train_sampler,
    collate_fn=processor.collate_fn,
    num_workers=4,  # num_worker = 4 * num_GPU
    pin_memory=True,
)
print("# train:", len(train_data))

dev_data = WOSDataset(dev_features)
dev_sampler = SequentialSampler(dev_data)
dev_loader = DataLoader(
    dev_data,
    batch_size=cfg.eval_batch_size,
    sampler=dev_sampler,
    collate_fn=processor.collate_fn,
    num_workers=4,
    pin_memory=True,
)
print("# dev:", len(dev_data))




# train: 46127
# dev: 5118


In [15]:
# Optimizer 및 Scheduler 선언
n_epochs = cfg.num_train_epochs

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

t_total = len(train_loader) * n_epochs
optimizer = AdamW(optimizer_grouped_parameters, lr=cfg.learning_rate, eps=cfg.adam_epsilon)
warmup_steps = int(t_total * cfg.warmup_ratio)
# learning rate decreases linearly from the initial lr set in the optimizer to 0
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
teacher_forcing = cfg.teacher_forcing_ratio

loss_fnc_1 = masked_cross_entropy_for_value  # generation
loss_fnc_2 = nn.CrossEntropyLoss()  # gating
loss_fnc_pretrain = nn.CrossEntropyLoss()  # MLM pretrain

In [16]:
# 모델 저장될 파일 위치 생성
if not os.path.exists(f"{cfg.model_dir}"):
    os.mkdir(f"{cfg.model_dir}")
if not os.path.exists(f"{cfg.model_dir}/{wandb.run.name}"):
    os.mkdir(f"{cfg.model_dir}/{wandb.run.name}")

In [17]:
json.dump(
    vars(cfg),
    open(f"{cfg.model_dir}/{wandb.run.name}/exp_config.json", "w"),
    indent=2,
    ensure_ascii=False,
)
json.dump(
    slot_meta,
    open(f"{cfg.model_dir}/slot_meta.json", "w"),
    indent=2,
    ensure_ascii=False,
)

### Pretraining

In [18]:
eval_data = json.load(open(f"../input/data/eval_dataset/eval_dials.json", "r"))

eval_examples = get_examples_from_dialogues(
    eval_data, user_first=False, dialogue_level=False
)

# Extracting Featrues
eval_features = processor.convert_examples_to_features(eval_examples)
eval_data = WOSDataset(eval_features)
eval_sampler = SequentialSampler(eval_data)
eval_loader = DataLoader(
    eval_data,
    batch_size=8,
    sampler=eval_sampler,
    collate_fn=processor.collate_fn,
)

100%|██████████| 2000/2000 [00:00<00:00, 15715.12it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


In [19]:
MLM_PRE = True

scaler = GradScaler()
n_pretrain_epochs = 10

def mlm_pretrain(loader, n_epochs):
    model.train()
    for step, batch in enumerate(tqdm(loader)):
        input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [b.to(device) if not isinstance(b, list) else b for b in batch]
        
        with autocast(): # 밑에 해당하는 코드를 자동으로 mixed precision으로 변환시켜서 실행
            logits, labels = model.forward_pretrain(input_ids, tokenizer)
            loss = loss_fnc_pretrain(logits.view(-1, config.vocab_size), labels.view(-1))

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()

        if step % 100 == 0:
            print('[%d/%d] [%d/%d] %f' % (epoch, n_epochs, step, len(loader), loss.item()))

if MLM_PRE:
    for epoch in range(n_pretrain_epochs):
        mlm_pretrain(eval_loader, n_pretrain_epochs)

  0%|          | 2/1847 [00:00<02:37, 11.70it/s]

[0/10] [0/1847] 10.665379


  6%|▌         | 102/1847 [00:08<02:21, 12.36it/s]

[0/10] [100/1847] 9.335777


 11%|█         | 202/1847 [00:16<02:17, 12.00it/s]

...

 92%|█████████▏| 1702/1847 [02:18<00:11, 12.83it/s]

[9/10] [1700/1847] 0.832226


 98%|█████████▊| 1802/1847 [02:26<00:03, 12.96it/s]

[9/10] [1800/1847] 1.297751


100%|██████████| 1847/1847 [02:30<00:00, 12.27it/s]


### Training

In [20]:
# backward pass시 gradient 정보가 손실되지 않게 하려고 사용(loss에 scale factor를 곱해서 gradient 값이 너무 작아지는 것을 방지)
scaler = GradScaler()
best_score, best_checkpoint = 0, 0

for epoch in range(n_epochs):
    start_time = time.time()
    batch_loss = []
    model.train()
    for step, batch in enumerate(train_loader):
        input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [
            b.to(device) if not isinstance(b, list) else b for b in batch
        ]
        # mask
        if cfg.mask:
            change_mask_prop = 0.8
            mask_p = random.random()
            if cfg.mask and mask_p < change_mask_prop:
                input_ids = custom_to_mask(input_ids)
        # teacher forcing
        if (
            teacher_forcing > 0.0
            and random.random() < teacher_forcing
        ):
            tf = target_ids
        else:
            tf = None

        optimizer.zero_grad()  # optimizer는 input으로 model parameter를 가진다 -> zero_grad()로 파라미터 컨드롤 가능

        with autocast():  # 밑에 해당하는 코드를 자동으로 mixed precision으로 변환시켜서 실행
            all_point_outputs, all_gate_outputs = model(
                input_ids, segment_ids, input_masks, target_ids.size(-1), tf
            )
            # generation loss
            loss_1 = loss_fnc_1(
                all_point_outputs.contiguous(),
                target_ids.contiguous().view(-1),
                tokenizer.pad_token_id,
            )
            # gating loss
            loss_2 = loss_fnc_2(
                all_gate_outputs.contiguous().view(-1, cfg.n_gate),
                gating_ids.contiguous().view(-1),
            )
            loss = loss_1 + loss_2
        batch_loss.append(loss.item())

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # global_step 추가 부분
        wandb.log({"train/learning_rate": get_lr(scheduler),
                   "train/epoch": epoch
                   })
        if step % 100 == 0:
            print(
                f"[{epoch}/{n_epochs}] [{step}/{len(train_loader)}] loss: {loss.item()} gen: {loss_1.item()} gate: {loss_2.item()}"
            )

            # -- train 단계에서 Loss, Accuracy 로그 저장
            wandb.log({
                "train/loss": loss.item(),
                "train/gen_loss": loss_1.item(),
                "train/gate_loss": loss_2.item(),
            })

    predictions, p_logits, p_idx, g_logits = inference(model, dev_loader, processor, device, cfg.n_gate)
    eval_result = _evaluation(predictions, dev_labels, slot_meta)

    # -- eval 단계에서 Loss, Accuracy 로그 저장
    wandb.log({
        "eval/join_goal_acc": eval_result["joint_goal_accuracy"],
        "eval/turn_slot_f1": eval_result["turn_slot_f1"],
        "eval/turn_slot_acc": eval_result["turn_slot_accuracy"],
    })

    for k, v in eval_result.items():
        print(f"{k}: {v}")

    if best_score < eval_result['joint_goal_accuracy']:
        cpprint(f"--Update Best checkpoint!, epoch: {epoch+1}")
        best_score = eval_result['joint_goal_accuracy']
        best_checkpoint = epoch
        if not os.path.isdir(cfg.model_dir):
            os.makedirs(cfg.model_dir)
        print("--Saving best model checkpoint")
        torch.save(model.state_dict(), f"{cfg.model_dir}/{wandb.run.name}/best.pth")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler.state_dict': scheduler.state_dict(),
            'loss': loss.item(),
            'gen_loss': loss_1.item(),
            'gate_loss': loss_2.item(),
        }, os.path.join(f"{cfg.model_dir}/{wandb.run.name}", "training_best_checkpoint.bin"))
        
        # logit 저장
        np.save(os.path.join(f"{cfg.model_dir}/{wandb.run.name}", r'p_logits.npy'), p_logits)
        np.save(os.path.join(f"{cfg.model_dir}/{wandb.run.name}", r'p_idx.npy'), p_idx)
        np.save(os.path.join(f"{cfg.model_dir}/{wandb.run.name}", r'g_logits.npy'), g_logits)

        
    torch.save(model.state_dict(), f"{cfg.model_dir}/{wandb.run.name}/last.pth")
    print(f"time for 1 epoch: {time.time() - start_time}")

[0/30] [0/11532] loss: 11.108061790466309 gen: 9.425432205200195 gate: 1.6826297044754028
[0/30] [100/11532] loss: 2.7578675746917725 gen: 2.32616925239563 gate: 0.4316982626914978
[0/30] [200/11532] loss: 2.347721815109253 gen: 1.716150164604187 gate: 0.6315717101097107
[0/30] [300/11532] loss: 2.3156487941741943 gen: 1.7092682123184204 gate: 0.6063806414604187
[0/30] [400/11532] loss: 2.379779577255249 gen: 1.5815484523773193 gate: 0.7982310652732849
[0/30] [500/11532] loss: 1.5391714572906494 gen: 1.0021620988845825 gate: 0.5370092988014221
[0/30] [600/11532] loss: 1.3581907749176025 gen: 0.846183717250824 gate: 0.5120069980621338
[0/30] [700/11532] loss: 1.8442423343658447 gen: 1.2138514518737793 gate: 0.6303908228874207
[0/30] [800/11532] loss: 1.0953497886657715 gen: 0.7124114036560059 gate: 0.382938414812088
[0/30] [900/11532] loss: 1.4426168203353882 gen: 0.9204893708229065 gate: 0.5221274495124817
[0/30] [1000/11532] loss: 1.4483888149261475 gen: 0.9628758430480957 gate: 0.485

100%|██████████| 640/640 [02:01<00:00,  5.28it/s]


{'joint_goal_accuracy': 0.30656506447831183, 'turn_slot_accuracy': 0.9659024792670887, 'turn_slot_f1': 0.8526494571044264}
joint_goal_accuracy: 0.30656506447831183
turn_slot_accuracy: 0.9659024792670887
turn_slot_f1: 0.8526494571044264
'--Update Best checkpoint!, epoch: 1'
--Saving best model checkpoint


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


time for 1 epoch: 2073.628895998001
[1/30] [0/11532] loss: 0.09361106157302856 gen: 0.06249314546585083 gate: 0.031117917969822884
[1/30] [100/11532] loss: 0.20754575729370117 gen: 0.15954691171646118 gate: 0.04799885302782059
[1/30] [200/11532] loss: 0.22125911712646484 gen: 0.15587329864501953 gate: 0.06538582593202591
[1/30] [300/11532] loss: 0.20811662077903748 gen: 0.18028445541858673 gate: 0.027832161635160446
[1/30] [400/11532] loss: 0.4564060568809509 gen: 0.2856338322162628 gate: 0.1707722246646881
[1/30] [500/11532] loss: 0.1410551518201828 gen: 0.12343443930149078 gate: 0.017620714381337166
[1/30] [600/11532] loss: 0.16064515709877014 gen: 0.1015995666384697 gate: 0.05904558673501015
[1/30] [700/11532] loss: 0.2400299608707428 gen: 0.22909818589687347 gate: 0.010931774973869324
[1/30] [800/11532] loss: 0.13029824197292328 gen: 0.10959453135728836 gate: 0.020703714340925217
[1/30] [900/11532] loss: 0.20407912135124207 gen: 0.1551753580570221 gate: 0.04890377074480057
[1/30] [

...

100%|██████████| 640/640 [01:59<00:00,  5.38it/s]


{'joint_goal_accuracy': 0.8151621727237202, 'turn_slot_accuracy': 0.9941730710781139, 'turn_slot_f1': 0.97498999566925}
joint_goal_accuracy: 0.8151621727237202
turn_slot_accuracy: 0.9941730710781139
turn_slot_f1: 0.97498999566925
time for 1 epoch: 1967.4125971794128
[28/30] [0/11532] loss: 6.876568659208715e-05 gen: 4.9007889174390584e-05 gate: 1.9757793779717758e-05
[28/30] [100/11532] loss: 5.688960663974285e-05 gen: 3.107321390416473e-05 gate: 2.5816390916588716e-05
[28/30] [200/11532] loss: 0.007588536944240332 gen: 0.00757877342402935 gate: 9.76329010882182e-06
[28/30] [300/11532] loss: 0.006140152923762798 gen: 0.006122842896729708 gate: 1.73100306710694e-05
[28/30] [400/11532] loss: 5.80010237172246e-05 gen: 5.373614840209484e-05 gate: 4.264875769877108e-06
[28/30] [500/11532] loss: 0.0006017561536282301 gen: 0.0005877602961845696 gate: 1.3995844710734673e-05
[28/30] [600/11532] loss: 0.007207480259239674 gen: 0.0071927872486412525 gate: 1.469305880164029e-05
[28/30] [700/11532]

100%|██████████| 640/640 [02:03<00:00,  5.17it/s]


{'joint_goal_accuracy': 0.8155529503712388, 'turn_slot_accuracy': 0.9941730710781143, 'turn_slot_f1': 0.9749586696482246}
joint_goal_accuracy: 0.8155529503712388
turn_slot_accuracy: 0.9941730710781143
turn_slot_f1: 0.9749586696482246
time for 1 epoch: 1978.1460721492767
[29/30] [0/11532] loss: 5.125667667016387e-05 gen: 3.9952679799171165e-05 gate: 1.1303997780487407e-05
[29/30] [100/11532] loss: 0.0003210804716218263 gen: 0.0003079766174778342 gate: 1.3103862329444382e-05
[29/30] [200/11532] loss: 0.00025593183818273246 gen: 0.00023967168817762285 gate: 1.6260142729151994e-05
[29/30] [300/11532] loss: 0.011781229637563229 gen: 0.011753227561712265 gate: 2.8002301405649632e-05


KeyboardInterrupt: 

In [23]:
!python inference.py

100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 6211.51it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
# eval: 14771
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Model is loaded
100%|███████████████████████████████████████| 1847/1847 [05:50<00:00,  5.27it/s]
